In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Dataframe to store data
df = pd.DataFrame(columns=['name', 'address', 'hours'])

url = 'https://www.afisha.uz/ru/catalog/restaurants?page='
driver = webdriver.Chrome()
for page in range(1, 49):
    driver.get(url + str(page))
    time.sleep(0.5)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    restaurants = soup.find_all('article', {'class': 'flex justify-between'})
    for restaurant in restaurants:
        name = restaurant.find('a', {'class': 'font-semibold hover:text-afisha transition-colors'}).text
        address = restaurant.find_all('div', {'class': 'text-gray-400 text-[13px] mt-1'})
        hours = address[2].text[7:] if len(address) > 1 else "No hours available"
        address = address[1].text if len(address) > 1 else "No address available"
        new_row = {'name': name, 'address': address, 'hours': hours}
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

df.to_csv('afisha_food.csv', index=False)

df = pd.read_csv('afisha_food.csv')
df.head()

In [36]:
food_delievery_points = pd.DataFrame(columns=['name', 'address', 'phone', 'categories', 'rating', 'amount_of_reviews', 'avg_check', 'price_eval', 'working_hours'])
reviews = pd.DataFrame(columns=['point_id', 'review', 'rating', 'date'])

In [37]:
driver = webdriver.Chrome()
driver.get('https://yandex.uz/maps/10335/tashkent/search/%D0%93%D0%B4%D0%B5%20%D0%BF%D0%BE%D0%B5%D1%81%D1%82%D1%8C/?ll=69.289531%2C41.319122&sll=69.364435%2C41.302741&sspn=0.615441%2C0.271155&z=12.68')
a = input('Press enter to continue')
switch_language = driver.find_element(By.CLASS_NAME, 'lang-switcher-view')
switch_language.click()
time.sleep(1)
russian = driver.find_elements(By.CLASS_NAME, 'lang-switcher-view__language')
russian[0].click()
time.sleep(1)

In [38]:
def scroll_to_bottom(driver, value, id, SCROLL_PAUSE_TIME):

    fBody = driver.find_elements(by=By.CSS_SELECTOR, value=value)[id]

    # Get scroll height
    last_height = driver.execute_script("return arguments[0].scrollHeight", fBody)
    while True:
        # driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;', fBody)
        driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", fBody)
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return arguments[0].scrollHeight", fBody)
        
        if new_height == last_height:
            break
        last_height = new_height

In [39]:
# Function to assign null if element not found
def find_element(driver, by, value):
    try:
        return driver.find_element(by=by, value=value).text
    except NoSuchElementException:
        return None

In [40]:
scroll_to_bottom(driver, ".scroll__container", 0, 2)
restaurants = driver.find_elements(by=By.CLASS_NAME, value='search-business-snippet-view__content')
for restaurant in restaurants:
    try:
        name, address, phone, categories, rating, amount_of_reviews, price_eval, avg_check, working_hours, avg_check_elements, price_eval = [None] * 11  # Initialize all values to None
        restaurant.click()
        time.sleep(2)
        name = find_element(driver, By.CLASS_NAME, 'card-title-view__title')
        address = find_element(driver, By.CLASS_NAME, 'business-contacts-view__address-link')
        phone = find_element(driver, By.CSS_SELECTOR, 'span[itemprop="telephone"]')
        categories = driver.find_elements(By.CSS_SELECTOR, '.business-card-title-view__category._outline')
        if categories:
            categories = [category.text for category in categories]
        else:
            categories = None
        rating = find_element(driver, By.CSS_SELECTOR, '.business-rating-badge-view__rating-text._size_m')
        amount_of_reviews = find_element(driver, By.CSS_SELECTOR, '.business-header-rating-view__text._clickable')
        avg_check_elements = driver.find_elements(by=By.CSS_SELECTOR, value='.business-features-view__valued-value')

        if avg_check_elements:
            price_eval = avg_check_elements[0].text
            try:
                avg_check = avg_check_elements[1].text
            except IndexError:
                avg_check = None
        else:
            price_eval = None
            avg_check = None

        working_hours = find_element(driver, By.CLASS_NAME, 'business-card-working-status-view__text')
        point_id = len(food_delievery_points) # index of the row to be added

        try:
            element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, ".tabs-select-view__title._name_reviews")))
            driver.execute_script("arguments[0].click();", element)
            time.sleep(1)
        except NoSuchElementException:
            print('No view more button')


        scroll_to_bottom(driver, ".scroll__container", 1, 0.5)
        reviews_elements = driver.find_elements(by=By.CLASS_NAME, value='business-review-view__info')
    except:
        print('Error occured in outer loop')
        continue
    food_delievery_points.loc[point_id] = [name, address, phone, categories, rating, amount_of_reviews, avg_check, price_eval, working_hours]
    for review_element in reviews_elements:
        try:
            # initialize all values to None
            review, rating, date = [None] * 3

            review = find_element(review_element, By.CLASS_NAME, 'business-review-view__body-text')
            try:
                rating = review_element.find_element(By.CSS_SELECTOR, "meta[itemprop='ratingValue']").get_attribute('content')
            except NoSuchElementException:
                rating = None
            try:
                date = review_element.find_element(By.CSS_SELECTOR, "meta[itemprop='datePublished']").get_attribute('content')
            except NoSuchElementException:
                date = None
        except:
            print('Error occured in inner loop')
            continue
        reviews.loc[len(reviews)] = [point_id, review, rating, date] # index is the index of the point in food_delievery_points

Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in outer loop
Error occured in out

In [41]:
food_delievery_points.head()

,name,address,phone,categories,rating,amount_of_reviews,avg_check,price_eval,working_hours
0,Подушечная,"Юнусабадский район, ул. Осиё, 17",+998 90 999 77 61,"[Ресторан, пиццерия, кафе]","4,4",434 отзыва,80000–120000 узб.сум,выше среднего,Открыто до 23:00
1,Nice Joy,"Юнусабадский район, ул. Осиё, 89/33",+998 90 999 77 62,"[Ресторан, суши-бар, кафе]","4,4",127 отзывов,80000–100000 узб.сум,выше среднего,Открыто до 23:00
2,Силк96,"ул. Кары-Ниязи, 68",+998 99 835 85 00,"[Ресторан, кафе, бар, паб]","4,4",80 отзывов,80000–120000 узб.сум,выше среднего,Закрыто до 15:00
3,Akmal Oltin Baliq,"Ташкентская область, Кибрайский район, улица У...",+998 97 145 49 95,[Кафе],"4,4",188 отзывов,None,None,Открыто до 23:00
4,СМИ,"ул. Кары-Ниязи, 68",+998 99 835 85 00,"[Бар, паб, кафе]","4,4",111 отзывов,высокие,30000–50000 узб.сум,Закрыто до 20:00


In [42]:
reviews.head()

,point_id,review,rating,date
0,0,"Еда на высоком уровне,подача еды для меня очен...",5.0,2023-05-18T13:09:36.880Z
1,0,"Уютное заведение, еда вкусная, цены приемлемые...",4.0,2023-08-29T20:25:05.994Z
2,0,"Отличное место! Кухня потрясающая, сотрудники ...",5.0,2023-10-16T07:49:00.514Z
3,0,Отличное заведение! Персонал просто душки!!!!😊...,5.0,2023-10-17T16:46:22.736Z
4,0,"Все понравилось, - и уютная обстановка, и вкус...",5.0,2023-05-25T16:44:29.717Z


In [45]:
food_delievery_points.shape

(73, 9)

In [47]:
reviews.shape

(3466, 4)

In [3]:
restaurants = pd.read_csv('food_delievery_points.csv')
reviews = pd.read_csv('reviews.csv')

In [9]:
reviews.head()

,point_id,review,rating,date
0,0,"Еда на высоком уровне,подача еды для меня очен...",5.0,2023-05-18T13:09:36.880Z
1,0,"Уютное заведение, еда вкусная, цены приемлемые...",4.0,2023-08-29T20:25:05.994Z
2,0,"Отличное место! Кухня потрясающая, сотрудники ...",5.0,2023-10-16T07:49:00.514Z
3,0,Отличное заведение! Персонал просто душки!!!!😊...,5.0,2023-10-17T16:46:22.736Z
4,0,"Все понравилось, - и уютная обстановка, и вкус...",5.0,2023-05-25T16:44:29.717Z


# find mean of ratings from reviews for each point and add it to food_delievery_points
# covert to float ratings
reviews['rating'] = reviews['rating'].astype(float)
reviews = reviews[['point_id', 'rating']]
mean_ratings = reviews.groupby(['point_id']).mean()
mean_ratings = mean_ratings.reset_index()
mean_ratings = mean_ratings[['point_id', 'rating']]
mean_ratings.head()